## Pseudo-code

In [4]:
# setup
# (x) randomly pick 300 words from ../vocab.txt (this will be given by Julia)
# (x) save that as a txt file, one line per word
# (x) generate a list containing 2500-3500ms (include start and end) in 250ms intervals
# (x) write a python function (inside) that 1) load 2) randomize and 3) return a list of 4 lists of 75 words
# (x) write a python function (inside) that randomly pick a interstimuli time

In [5]:
import time
import keyboard
import numpy as np
import sys
sys.path.append('../scripts/')
import time_util as tu
from psychopy.hardware import keyboard
import ipywidgets as widgets
import IPython.display as ipd
import threading

pygame 2.0.0.dev3 (SDL 2.0.9, python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Setup

In [6]:
kNUM_WORDS_PER_COND = 75
kNUM_CONDS = 4

Tasks:
- randomly pick 300 words from ../vocab.txt (this will be given by Julia)
- save that as a txt file, one line per word

In [7]:
# with open('../vocab.txt', 'r') as txt_f:
#     words = [word.rstrip() for word in txt_f.readlines()]

In [8]:
# np.random.seed(1)
# sdt_words = np.random.choice(words, size=300)

In [9]:
with open('sdt_vocab.txt', 'w+') as txt_f:
    for word in sdt_words:
        txt_f.write(word + '\n')

In [10]:
def get_word_lists(sdt_vocab_fpath):
    with open(sdt_vocab_fpath, 'r') as txt_f:
        words = [word.rstrip() for word in txt_f.readlines()]
    np.random.shuffle(words)
    return np.split(np.array(words), kNUM_CONDS)

In [11]:
word_lists = get_word_lists('sdt_vocab.txt')

Tasks:
- generate a list containing 2500-3500ms (include start and end) in 250ms intervals
- save that as a json file

In [12]:
def get_interstim_intervals_for_all(size=300):
    choices = np.arange(2500, 3500 + 1, 250)
    return np.random.choice(choices, size=size)

In [13]:
get_interstim_intervals_for_all()[:10]

array([2750, 3000, 3500, 3250, 3000, 3250, 3250, 3000, 3250, 2500])

In [14]:
len(get_interstim_intervals_for_all())

300

## Experiment

In [15]:
# experiment:
# kNUM_WORDS_PER_COND = 75
# interstim_intervals = get_interstim_interval_for_all(size=300)
# for i, condition in enumerate(randomize(noise_conditions)):
#     word_list = word_lists[idx]
#     for j, word in enumerate(word_list):
#         print(word) <- timer starts
#         make noun judgment <- takes no time
#         record word presented, present time, interstimulus interval, response time <- takes no time
#         time.sleep(interstim_intervals[i * kNUM_WORDS_PER_COND + j])

In [16]:
class Clock:
    
    @property
    def now(self):
        return time.time()

In [17]:
class Timer:
    
    def __init__(self):
        self.start, self.end = None, None
        
    def record_start(self, time):
        self.start = time
        
    def record_end(self, time):
        self.end = time
        
    def get_duration(self):
        return self.end - self.start

In [18]:
class TimerFactory:
    
    def get_new_timer(self):
        return Timer()

In [32]:
class Experiment:
    
    def __init__(self):
        self._running = True
        self.resps = []
        
    def is_noun(self, temp):
        self.resps.append(1)
        print('Response time:', round(self.clock.now - self.timer.start, 3), 'seconds')
        
    def terminate(self):
        self._running = False
    
    def run(self, button):

        self.clock = Clock()
        timer_factory = TimerFactory()
        timers = []

        default_keyboard = keyboard.Keyboard()

        noise_conds = ['1', '2', '3', '4']
        interstim_intervals = get_interstim_intervals_for_all(size=300)

        for i, cond in enumerate(noise_conds):
            word_list = word_lists[i]
            for j, word in enumerate(word_list):
                
                if self._running:

                    resps_len_before = len(self.resps)

                    max_resp_time = interstim_intervals[i * kNUM_WORDS_PER_COND + j]

                    self.timer = timer_factory.get_new_timer()
                    self.timer.start = self.clock.now

                    print(word)
                    ipd.display(button)
                    time.sleep(max_resp_time / 1000)
                    ipd.clear_output()

                    resps_len_after = len(self.resps)

                    if resps_len_after == resps_len_before:
                        self.resps.append(0)
                    elif resps_len_after - resps_len_before >= 2:
                        self.resps = self.resps[:resps_len_before+1]

                    print('Responses:', self.resps)

In [36]:
sdt = Experiment()

button = widgets.Button(
    description='This is a noun!',
    layout={'width': '300px'}
)
button.on_click(sdt.is_noun)

thread = threading.Thread(target=sdt.run, args=(button, ))
thread.start()

Responses: [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
magazines


Button(description='This is a noun!', layout=Layout(width='300px'), style=ButtonStyle())

In [38]:
sdt.terminate()
thread.join()